In [27]:
# project: p10
# submitter: bai59
# partner: none
# hours: 2

In [28]:
import os
import csv
import json
from collections import namedtuple
# you may use from style of import to import JSONDecodeError

In [29]:
def list_files_in(pathname):
    res = []
    for i in os.listdir(pathname):
        if(i[0]!='.'):
            res.append(i)
    sorted(res)
    return res

In [30]:
#q1
list_files_in("data")

['channel_ids1.json',
 'channel_ids2.json',
 'channel_ids3.json',
 'channel_ids4.json',
 'channel_ids5.json',
 'comment_data1.csv',
 'comment_data2.csv',
 'comment_data3.csv',
 'comment_data4.csv',
 'comment_data5.csv',
 'video_data.csv',
 'video_ids.json']

In [31]:
def list_paths_in(pathname):
    return [os.path.join(pathname,i) for i in list_files_in(pathname)]

In [32]:
#q2
list_paths_in_data = list_paths_in("data")
list_paths_in_data

['data\\channel_ids1.json',
 'data\\channel_ids2.json',
 'data\\channel_ids3.json',
 'data\\channel_ids4.json',
 'data\\channel_ids5.json',
 'data\\comment_data1.csv',
 'data\\comment_data2.csv',
 'data\\comment_data3.csv',
 'data\\comment_data4.csv',
 'data\\comment_data5.csv',
 'data\\video_data.csv',
 'data\\video_ids.json']

In [33]:
#q3
comment_paths = [i for i in list_paths_in_data if 'comment_data' in i]
comment_paths.sort()
comment_paths

['data\\comment_data1.csv',
 'data\\comment_data2.csv',
 'data\\comment_data3.csv',
 'data\\comment_data4.csv',
 'data\\comment_data5.csv']

In [34]:
#q4
channel_paths = [i for i in list_paths_in_data if 'channel_ids' in i]
channel_paths.sort()
channel_paths

['data\\channel_ids1.json',
 'data\\channel_ids2.json',
 'data\\channel_ids3.json',
 'data\\channel_ids4.json',
 'data\\channel_ids5.json']

In [35]:
def read_json(path):
    with open(path, encoding="utf-8") as f:
        return json.load(f) # dict, list, etc
def get_mapping(pathname):
    try:
        return read_json(pathname)
    except json.JSONDecodeError:
        return {}

In [36]:
#q5
get_mapping(os.path.join("data", "channel_ids1.json"))['UCNye-wNBqNL5ZzHSJj3l8Bg']

'Al Jazeera English'

In [37]:
#q6
channel_dict = {}
counter = 0
for i in channel_paths:
    for j in get_mapping(i).items():
        channel_dict[j[0]] = j[1]
        counter += 1
counter

372

In [38]:
Comment = namedtuple("Comment",["video_id","comment_length","author_id","likes","published_at"])

In [39]:
# You will need process_csv in order to complete get_comment_data.
# If you already copy/pasted it from lab, please ignore this copy.
def process_csv(filename):
    exampleFile = open(filename, encoding="utf-8")  
    exampleReader = csv.reader(exampleFile)
    exampleData = list(exampleReader)        
    exampleFile.close()  
    return exampleData

def get_comment_data(comment_file):
    csv_data = process_csv(comment_file)
    header = csv_data[0]
    comment_rows = csv_data[1:]
    comment_id_idx = header.index("comment_id")
    video_id_idx = header.index("video_id")
    comment_length_idx = header.index("comment_length")
    author_id_idx = header.index("author_id")
    likes_idx = header.index("likes")
    published_at_idx = header.index("published_at")
    # Iterate over comment_rows
    res = {}
    for i in comment_rows:
        if len(i) != 6 or '' in i:
            continue
        try:
            res[i[comment_id_idx]] = Comment(i[video_id_idx],int(i[comment_length_idx]),i[author_id_idx],int(i[likes_idx]),i[published_at_idx])
        except ValueError:
            continue
    return res

In [40]:
#q7
#What is the Comment object with comment ID UgygOezB4Mvd5o6FgAt4AaABAg
get_comment_data(os.path.join("data", "comment_data1.csv"))['UgygOezB4Mvd5o6FgAt4AaABAg']

Comment(video_id='udNXMAflbU8', comment_length=175, author_id='UCHkk7x38KWgqjQOHqsQwf0Q', likes=47, published_at='2021-10-10 17:48:38')

In [41]:
comments = {}
for file in comment_paths:
    comments.update(get_comment_data(file))

In [42]:
#q8
#What is the length of the comment with ID UgztIaGfqFoiGvbOdfp4AaABAg
comments['UgztIaGfqFoiGvbOdfp4AaABAg'].comment_length

67

In [43]:
#q9
#What percentage of comments are at most 140 characters long?
len([i for i in comments.values() if i.comment_length<=140])/len(comments) * 100

86.86953042956443

In [44]:
#q10
#What is the author ID of the comment that has the highest number of likes?
max_val = -1
for i in comments.values():
    if i.likes>max_val:
        max_val = i.likes
        max_id = i.author_id
max_id

'UCIPPMRA040LQr5QPyJEbmXA'

In [45]:
#Q11
#calculate the most active hour for posting comment
# dict of 24 hours
hour_dict = {i:0 for i in range(24)}
for i in comments.values():
    hour_dict[int(i.published_at[11:13])] += 1
sorted(hour_dict.items(), key=lambda x: x[1], reverse=True)[0][0]

19

In [46]:
# Bucketize the comments data by creating a dict mapping video IDs to a list of comment IDs corresponding to that video ID.
comment_buckets = {}
for i in comments:
    if comments[i].video_id not in comment_buckets:
        comment_buckets[comments[i].video_id] = [i]
    else:
        comment_buckets[comments[i].video_id].append(i)

In [47]:
#q12
# How many comments does the video with ID A8rrr_w8rfk have?
len(comment_buckets['A8rrr_w8rfk'])

606

In [63]:
def get_videos(data_file, video_mapping_file):
    """
    Given data_file (csv file) and video_mapping_file (json) file, generates a video
    dictionary, mapping video ID to a dictionary containing title, and other details of the video.
    Handles missing entry in video_ids.json by using try / except blocks to handle KeyError.
    """
    data = process_csv(data_file)
    header = data[0]
    all_videos = data[1:]
    video_mapping = get_mapping(video_mapping_file)
    videos_dict = dict()
    for video in all_videos: # You may find it helpful to do all_videos[:5] to only look at the first 5 videos.
        try:
            videos_dict[video[header.index("video_id")]] = {
                "title": video_mapping[video[header.index("video_id")]],
                "channel_id": video[header.index("channel_id")],
                "published_at": video[header.index("published_at")],
                "duration": video[header.index("duration")],
                "category": video[header.index("category")],
                "tags": video[header.index("tags")],
                "views": int(video[header.index("views")]),
                "likes": int(video[header.index("likes")]),
                "dislikes": int(video[header.index("dislikes")]),
            }
        except KeyError:
            continue
        except ValueError:
            continue
    return videos_dict
videos = get_videos(os.path.join('data','video_data.csv'), os.path.join('data','video_ids.json'))